In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')

In [3]:
import time
import pandas as pd
import numpy as np
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
from collections import deque
from itertools import islice
from collections import OrderedDict
from PrefixTreeCDD.PrefixTreeClass import PrefixTree
import PrefixTreeCDD.settings as settings
from PrefixTreeCDD.CDD import Window
from skmultiflow.drift_detection import ADWIN, PageHinkley
import math
from numpy import log as ln
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import utils as ku
import numpy.random as rn

In [4]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0.5
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
# d.create_batch("normal", timeformat)
is_written = 0

start_time = time.time()
# print(d.train.contextdata)
print("Test Context Data")
print(d.test_orig.contextdata)
basic_model = m.train(d.train)

Num events is 21348
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 10674
Test: 10674
Test Context Data
       level_0  index  event_Prev9  role_Prev9 completeTime_Prev9  case_Prev9  \
0        10674   7227            0           0                  0        1533   
1        10675    440            0           0                  0          93   
2        10676  11541            0           0                  0        2460   
3        10677  11938            0           0                  0        2546   
4        10678   9814            0           0                  0        2090   
...        ...    ...          ...         ...                ...         ...   
10669    21343    844            0           0                  0         176   
10670    21344  10513            0           0                  0        2240   
10671    21345  14862            0           0    

2023-05-18 13:13:27.014937: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 13:13:27.015929: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
event_Prev0 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev1 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev2 (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_Prev3 (InputLayer)        [(None, 1)]          0                                            
______________________________________________________________________________________________

In [5]:
from edbn.Methods.SDL.sdl import transform_data

In [6]:
[a for a in log.attributes() if a != log.time and a != log.trace]

['event',
 'startTime',
 'creator',
 'variant',
 'variant-index',
 'org:resource',
 'role',
 'Activity']

In [9]:
d.train.values

{'event': array(['Assign seriousness', 'Closed', 'Create SW anomaly', 'DUPLICATE',
        'INVALID', 'Insert ticket', 'RESOLVED', 'Require upgrade',
        'Resolve SW anomaly', 'Resolve ticket', 'Schedule intervention',
        'Take in charge ticket', 'VERIFIED', 'Wait'], dtype=object),
 'role': array(['Value 1', 'Value 10', 'Value 11', 'Value 12', 'Value 13',
        'Value 14', 'Value 15', 'Value 16', 'Value 17', 'Value 18',
        'Value 19', 'Value 2', 'Value 20', 'Value 21', 'Value 22',
        'Value 3', 'Value 4', 'Value 5', 'Value 6', 'Value 7', 'Value 8',
        'Value 9'], dtype=object),
 'case': array(['Case1', 'Case10', 'Case100', ..., 'Case997', 'Case998', 'Case999'],
       dtype=object)}

In [12]:
x,y,val=transform_data(d.logfile,[a for a in d.logfile.attributes() if a != d.logfile.time and a != d.logfile.trace])

In [33]:
d.logfile.attributes()

Index(['event', 'role', 'completeTime', 'case'], dtype='object')

In [40]:
d.logfile.data

,event,role,completeTime,case
0,1,12,2010/01/13 07:40:25.000,2900
1,1,12,2010/01/13 11:26:04.000,1944
2,1,12,2010/01/13 11:30:37.000,3651
3,1,12,2010/01/13 12:09:31.000,596
4,1,12,2010/01/13 16:25:25.000,3402
...,...,...,...,...
21343,2,16,2014/01/03 09:02:38.000,176
21344,2,16,2014/01/03 09:03:58.000,2240
21345,2,16,2014/01/03 09:08:18.000,3176
21346,2,16,2014/01/03 12:19:48.000,1763


In [36]:
d.logfile.contextdata

,index,event_Prev9,role_Prev9,completeTime_Prev9,case_Prev9,event_Prev8,role_Prev8,completeTime_Prev8,case_Prev8,event_Prev7,...,completeTime_Prev1,case_Prev1,event_Prev0,role_Prev0,completeTime_Prev0,case_Prev0,event,role,completeTime,case
0,13579,0,0,0,2900,0,0,0,2900,0,...,0,2900,0,0,0,2900,1,12,2010/01/13 07:40:25.000,2900
1,9143,0,0,0,1944,0,0,0,1944,0,...,0,1944,0,0,0,1944,1,12,2010/01/13 11:26:04.000,1944
2,17033,0,0,0,3651,0,0,0,3651,0,...,0,3651,0,0,0,3651,1,12,2010/01/13 11:30:37.000,3651
3,2842,0,0,0,596,0,0,0,596,0,...,0,596,0,0,0,596,1,12,2010/01/13 12:09:31.000,596
4,15901,0,0,0,3402,0,0,0,3402,0,...,0,3402,0,0,0,3402,1,12,2010/01/13 16:25:25.000,3402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21343,844,0,0,0,176,0,0,0,176,0,...,2013/11/20 08:10:39.000,176,10,12,2013/12/19 09:02:24.000,176,2,16,2014/01/03 09:02:38.000,176
21344,10513,0,0,0,2240,0,0,0,2240,0,...,2013/11/20 16:10:46.000,2240,10,12,2013/12/19 09:03:46.000,2240,2,16,2014/01/03 09:03:58.000,2240
21345,14862,0,0,0,3176,0,0,0,3176,0,...,2013/11/13 12:28:39.000,3176,10,12,2013/12/19 09:08:02.000,3176,2,16,2014/01/03 09:08:18.000,3176
21346,8300,0,0,0,1763,0,0,0,1763,0,...,2013/11/25 14:12:40.000,1763,10,1,2013/12/19 12:19:27.000,1763,2,16,2014/01/03 12:19:48.000,1763


In [28]:
list(map(lambda arr:np.unique(arr),x))

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14]),
 array([ 0,  1,  3,  6,  8,  9, 10, 11, 12, 14]),
 array([ 0,  1,  3,  6,  8, 10, 11, 12, 14]),
 array([ 0,  1,  3,  6,  8, 10, 12, 14]),
 array([ 0,  1,  3,  6, 10, 12, 14]),
 array([ 0,  1,  6, 10, 12, 14]),
 array([ 0,  1, 10, 12, 14]),
 array([ 0,  1, 10, 12, 14]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 15, 16, 17, 18,
        19, 20, 21, 22]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 18, 19, 20,
        21, 22])

In [29]:
tempdf=pd.DataFrame(x).T

In [49]:
attrs=[a for a in d.logfile.attributes() if a != d.logfile.time and a != d.logfile.trace]
cols=[]
for attr in attrs:
  for k in range(10):
    cols.append(attr + "_Prev%i" % k)
cols=sorted(cols)

In [50]:
cols

['event_Prev0',
 'event_Prev1',
 'event_Prev2',
 'event_Prev3',
 'event_Prev4',
 'event_Prev5',
 'event_Prev6',
 'event_Prev7',
 'event_Prev8',
 'event_Prev9',
 'role_Prev0',
 'role_Prev1',
 'role_Prev2',
 'role_Prev3',
 'role_Prev4',
 'role_Prev5',
 'role_Prev6',
 'role_Prev7',
 'role_Prev8',
 'role_Prev9']

In [51]:
tempdf.columns=cols

In [53]:
tempdf.T

,0,1,2,3,4,5,6,7,8,9,...,21338,21339,21340,21341,21342,21343,21344,21345,21346,21347
event_Prev0,0,0,0,0,0,1,0,1,0,0,...,10,10,10,10,10,10,10,10,10,10
event_Prev1,0,0,0,0,0,0,0,0,0,0,...,14,14,14,14,14,14,14,14,14,14
event_Prev2,0,0,0,0,0,0,0,0,0,0,...,12,14,12,12,12,12,12,12,12,12
event_Prev3,0,0,0,0,0,0,0,0,0,0,...,1,12,0,1,1,1,12,1,1,1
event_Prev4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
event_Prev5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
event_Prev6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
event_Prev7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
event_Prev8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
event_Prev9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
test_input=tf.keras.Input((1,))
y = tf.keras.layers.Dense(16, activation='softmax')(test_input)
test_model = tf.keras.Model(test_input, y)

In [95]:
test_x=np.array([1,2,3])

In [96]:
test_model.predict(test_x)

1/1 [==============================] - 0s 24ms/step


array([[0.04368756, 0.09770251, 0.05335451, 0.07677803, 0.1069745 ,
        0.0930449 , 0.05597579, 0.05664876, 0.06234029, 0.05038038,
        0.04260708, 0.06596675, 0.03571239, 0.04119217, 0.0566236 ,
        0.06101073],
       [0.02760833, 0.13808167, 0.04117815, 0.08527045, 0.16553323,
        0.1252304 , 0.04532368, 0.04642004, 0.05621629, 0.03671534,
        0.0262596 , 0.06294696, 0.01844856, 0.02454449, 0.04637882,
        0.05384396],
       [0.01583398, 0.17710632, 0.02884232, 0.08594643, 0.23246512,
        0.15296592, 0.03330563, 0.03452139, 0.04600694, 0.02428295,
        0.01468799, 0.054512  , 0.00864915, 0.01327275, 0.03447542,
        0.04312564]], dtype=float32)

In [73]:
basic_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 event_Prev0 (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 event_Prev1 (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 event_Prev2 (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 event_Prev3 (InputLayer)       [(None, 1)]          0           []                               
                                                                                              

In [98]:
basic_model.predict([np.array([1,2]) for i in range(20)])

1/1 [==============================] - 0s 31ms/step


array([[0.00474054, 0.07392824, 0.02120817, 0.00505907, 0.00424803,
        0.00960089, 0.00777785, 0.00596985, 0.00456289, 0.00483682,
        0.01595247, 0.00656548, 0.79402393, 0.00938819, 0.03213755],
       [0.00959098, 0.00710817, 0.86183333, 0.00653246, 0.0086733 ,
        0.00976765, 0.00693736, 0.02381342, 0.0090532 , 0.00736758,
        0.01177022, 0.00572224, 0.00719686, 0.02219619, 0.00243707]],
      dtype=float32)

In [103]:
print([np.array([1,2]) for i in range(20)])

[array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2]), array([1, 2])]


In [105]:
len(x[0])

21348

In [5]:
basic_model.save("basic_model")

2023-05-18 13:14:46.167544: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: basic_model/assets


In [6]:
temp=tf.keras.models.load_model("basic_model")

ValueError: Unknown entries in loss dictionary: ['output']. Only expected following keys: ['output_1']

In [110]:
temp